# L1 正则化（Lasso）

## 核心思想

L1 正则化通过在损失函数中添加权重绝对值之和的惩罚项，促使模型学习稀疏特征。

## 数学表达

```
Loss_total = Loss_original + λ × Σ|wᵢ|
```

其中 λ 是正则化强度超参数。

## 特点

| 特性 | 说明 |
|------|------|
| 稀疏性 | 倾向于将不重要的权重压缩为0 |
| 特征选择 | 自动进行特征选择 |
| 抗噪声 | 对噪声特征更鲁棒 |

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. 基本用法

In [ ]:
# L1 正则化层
# l1(0.01) 表示正则化系数 λ = 0.01
layer_l1 = keras.layers.Dense(
    units=64,
    activation='relu',
    kernel_regularizer=keras.regularizers.l1(0.01),
    kernel_initializer='he_normal'
)

print("L1 正则化层创建成功")
print(f"正则化系数: 0.01")

## 2. 完整模型示例

In [ ]:
# 构建带 L1 正则化的模型
model_l1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu', 
                      kernel_regularizer=keras.regularizers.l1(0.001),
                      kernel_initializer='he_normal'),
    keras.layers.Dense(128, activation='relu',
                      kernel_regularizer=keras.regularizers.l1(0.001),
                      kernel_initializer='he_normal'),
    keras.layers.Dense(10, activation='softmax')
])

model_l1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_l1.summary()

## 3. 可视化 L1 正则化效果

In [ ]:
def visualize_l1_effect():
    """
    可视化 L1 正则化的稀疏性效果
    """
    # 创建简单数据
    np.random.seed(42)
    X = np.random.randn(1000, 20)
    # 只有前5个特征是有用的
    y = X[:, :5].sum(axis=1) + np.random.randn(1000) * 0.1
    y = (y > y.mean()).astype(int)
    
    # 无正则化模型
    model_no_reg = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(20,)),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model_no_reg.compile(optimizer='adam', loss='binary_crossentropy')
    model_no_reg.fit(X, y, epochs=50, verbose=0)
    
    # L1 正则化模型
    model_l1_reg = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(20,),
                          kernel_regularizer=keras.regularizers.l1(0.01)),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model_l1_reg.compile(optimizer='adam', loss='binary_crossentropy')
    model_l1_reg.fit(X, y, epochs=50, verbose=0)
    
    # 获取第一层权重
    weights_no_reg = model_no_reg.layers[0].get_weights()[0]
    weights_l1 = model_l1_reg.layers[0].get_weights()[0]
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].imshow(np.abs(weights_no_reg), cmap='hot', aspect='auto')
    axes[0].set_title('无正则化 - 权重热图')
    axes[0].set_xlabel('输出神经元')
    axes[0].set_ylabel('输入特征')
    axes[0].colorbar = plt.colorbar(axes[0].images[0], ax=axes[0])
    
    axes[1].imshow(np.abs(weights_l1), cmap='hot', aspect='auto')
    axes[1].set_title('L1 正则化 - 权重热图（更稀疏）')
    axes[1].set_xlabel('输出神经元')
    axes[1].set_ylabel('输入特征')
    axes[1].colorbar = plt.colorbar(axes[1].images[0], ax=axes[1])
    
    plt.tight_layout()
    plt.show()
    
    # 统计接近零的权重比例
    threshold = 0.01
    sparse_no_reg = np.sum(np.abs(weights_no_reg) < threshold) / weights_no_reg.size
    sparse_l1 = np.sum(np.abs(weights_l1) < threshold) / weights_l1.size
    
    print(f"\n接近零的权重比例 (阈值={threshold}):")
    print(f"无正则化: {sparse_no_reg:.2%}")
    print(f"L1 正则化: {sparse_l1:.2%}")

visualize_l1_effect()

## 4. 正则化系数选择

In [ ]:
# 不同正则化强度的对比
l1_values = [0.0, 0.0001, 0.001, 0.01, 0.1]

print("L1 正则化强度选择建议:")
print("="*50)
print(f"{'λ 值':<10} {'效果':<40}")
print("-"*50)
print(f"{'0.0001':<10} {'轻微正则化，适合已经不太过拟合的模型':<40}")
print(f"{'0.001':<10} {'中等正则化，常用起始值':<40}")
print(f"{'0.01':<10} {'较强正则化，明显稀疏效果':<40}")
print(f"{'0.1':<10} {'强正则化，可能导致欠拟合':<40}")

In [ ]:
# 验证代码正确性
print("L1 正则化模块测试完成")
print("\n关键要点:")
print("1. L1 正则化产生稀疏权重（特征选择）")
print("2. 正则化系数 λ 控制稀疏程度")
print("3. 常用于需要特征选择的场景")